<a href="https://colab.research.google.com/github/gladcolor/homophily/blob/master/Homophily_Huan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install geopandas

     |████████████████████████████████| 994 kB 12.3 MB/s 
     |████████████████████████████████| 15.4 MB 80 kB/s 
     |████████████████████████████████| 6.3 MB 61.0 MB/s 


In [3]:
% cd /content/drive/MyDrive/Research/Homophily
import pandas as pd
from zipfile import ZipFile
import requests
from urllib.request import urlopen


MSA_name = r'Charlotte'

MSA_zip_url = f'https://github.com/gladcolor/homophily/raw/master/zip_files/{MSA_name}.zip'

zipresp = urlopen(MSA_zip_url)
tempzip = open(f"{MSA_name}.zip", "wb")
tempzip.write(zipresp.read())


MSA_edges_csv = f'{MSA_name}_2019_total_OD_travels_directed.csv'
MSA_nodes_csv = f'nodes.csv'

MSA_edges_df = pd.read_csv(ZipFile(f"{MSA_name}.zip").open(MSA_edges_csv), header=None, names=['origin', 'destination', 'trips'])
MSA_nodes_df = pd.read_csv(ZipFile(f"{MSA_name}.zip").open(MSA_nodes_csv), header=None, names=['tract', 'lat', 'lon'])


/content/drive/MyDrive/Research/Homophily


In [4]:
MSA_nodes_df = pd.read_csv(ZipFile(f"{MSA_name}.zip").open(MSA_nodes_csv))

MSA_nodes_df

,ID,Long,Lat
0,37119005843,-80.816466,35.038595
1,37119005916,-80.911822,35.132700
2,37119003808,-80.884576,35.145080
3,37119005918,-80.979934,35.090514
4,37119005914,-80.968109,35.121979
...,...,...,...
540,45091061707,-81.067355,35.124702
541,45091061003,-81.010646,35.040851
542,45091061006,-80.948944,35.083198
543,45091061801,-81.216258,35.124643


In [5]:
MSA_df = MSA_edges_df.merge(MSA_nodes_df, how='left', left_on="origin", right_on="ID" ).drop(columns=['ID'])
MSA_df = MSA_df.rename({"Long":"O_lon", "Lat":"O_lat"}, axis="columns")
MSA_df = MSA_df.merge(MSA_nodes_df, how='left', left_on="destination", right_on="ID" ).drop(columns=['ID'])
MSA_df = MSA_df.rename({"Long":"D_lon", "Lat":"D_lat"}, axis="columns")
MSA_df

,origin,destination,trips,O_lon,O_lat,D_lon,D_lat
0,37119003103,37119002800,788,-80.861828,35.167207,-80.821300,35.192552
1,37119003103,37119006007,93,-80.861828,35.167207,-80.976666,35.321815
2,37119003103,37119005509,130,-80.861828,35.167207,-80.802181,35.364325
3,37119003103,37179020307,123,-80.861828,35.167207,-80.630828,35.031372
4,37119003103,37119003802,2767,-80.861828,35.167207,-80.888691,35.189816
...,...,...,...,...,...,...,...
280070,37097060702,37119001911,1,-80.768287,35.841144,-80.718501,35.188613
280071,37109070201,37025041602,1,-81.229531,35.491678,-80.463345,35.342708
280072,37071030700,37179021012,1,-81.376690,35.365509,-80.726077,34.956224
280073,45057011001,37159051201,1,-80.676238,34.703903,-80.561587,35.625116


In [11]:
import geopy.distance
import os

def get_geo_distance(row):
    lat1 = row['O_lat']
    lon1 = row['O_lon']
    lat2 = row['D_lat']
    lon2 = row['D_lon']
    point1 = (lat1, lon1)
    point2 = (lat2, lon2)

    distance = geopy.distance.geodesic(point1, point2).m
    return distance

coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

# print(geopy.distance.vincenty(coords_1, coords_2).km)
# print(geopy.distance.geodesic(coords_1, coords_2).km)

print("Computing distances, may need 10 minites for a MAS...")


if not os.path.exists(distance_csv):
    distance_csv = f'{MSA_name}_distance.csv'
    MSA_df['geo_distance'] = MSA_df.apply(get_geo_distance, axis=1)
    MSA_df.to_csv(distance_csv, index=False)
    print("Done.")
else:
    MSA_df = pd.read_csv(distance_csv)



MSA_df

Computing distances, may need 10 minites for a MAS...


,origin,destination,trips,O_lon,O_lat,D_lon,D_lat,geo_distance
0,37119003103,37119002800,788,-80.861828,35.167207,-80.821300,35.192552,4640.554944
1,37119003103,37119006007,93,-80.861828,35.167207,-80.976666,35.321815,20086.574308
2,37119003103,37119005509,130,-80.861828,35.167207,-80.802181,35.364325,22532.726934
3,37119003103,37179020307,123,-80.861828,35.167207,-80.630828,35.031372,25898.152094
4,37119003103,37119003802,2767,-80.861828,35.167207,-80.888691,35.189816,3504.147790
...,...,...,...,...,...,...,...,...
280070,37097060702,37119001911,1,-80.768287,35.841144,-80.718501,35.188613,72539.087997
280071,37109070201,37025041602,1,-81.229531,35.491678,-80.463345,35.342708,71522.503263
280072,37071030700,37179021012,1,-81.376690,35.365509,-80.726077,34.956224,74669.471192
280073,45057011001,37159051201,1,-80.676238,34.703903,-80.561587,35.625116,102734.961566


In [12]:
MSA_df['geo_distance'].min()

535.8383656005901

$ T_{A, B} = {k} \frac{{(Pop_A)}^{\alpha} {(Pop_B)}^{\beta}}{{(Dist_{AB})}^{\zeta}}  $ 

In [104]:
ACS2019_csv = r'https://github.com/gladcolor/homophily/raw/master/ACS_race_cbsa_income_2019.zip'

ACS2019_df = pd.read_csv(ACS2019_csv)
ACS2019_df['tract'] = ACS2019_df['GEOID_Data'].str[-12:-1].astype(int)
ACS2019_df.sample(3)

,OBJECTID,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,INTPTLAT,INTPTLON,Shape_Area,GEOID_Data,county_code,cbg_total_population,p_asian,p_black,p_white,CBSA Code,CSA Code,CBSA Title,Metropolitan/Micropolitan Statistical Area,CSA Title,County/County Equivalent,State Name,FIPS State Code,FIPS County Code,GEOID,total_household_income,total_households,mean_household_income,median_household_income,tract
83910,90720.0,25,17,374000,2,42.299610,-71.199369,3.925652e+06,15000US250173740002,25017,2206,0.117860,0.026292,0.794651,14460,148.0,"Boston-Cambridge-Newton, MA-NH",Metropolitan Statistical Area,"Boston-Worcester-Providence, MA-RI-NH-CT",Middlesex County,Massachusetts,25,17,15000US250173740002,77694817.5,637.0,121969.886185,131625,25017374000
70357,74588.0,19,153,10703,3,41.652020,-93.481059,6.439145e+05,15000US191530107033,19153,1580,0.005063,0.000000,0.972785,19780,218.0,"Des Moines-West Des Moines, IA",Metropolitan Statistical Area,"Des Moines-Ames-West Des Moines, IA",Polk County,Iowa,19,153,15000US191530107033,57804719.5,616.0,93838.830357,79318,19153010703
104140,111812.0,29,95,14002,2,38.953839,-94.163813,2.718694e+07,15000US290950140022,29095,1070,0.000000,0.000000,1.000000,28140,312.0,"Kansas City, MO-KS",Metropolitan Statistical Area,"Kansas City-Overland Park-Kansas City, MO-KS",Jackson County,Missouri,29,95,15000US290950140022,29687306.5,463.0,64119.452484,44438,29095014002


In [108]:
ACS2019_tract_pop = ACS2019_df.groupby(['tract']).agg(Tract_pop = ("cbg_total_population", 'sum')).reset_index()
ACS2019_tract_pop

,tract,Tract_pop
0,1001020100,1993
1,1001020200,1959
2,1001020300,3507
3,1001020400,3878
4,1001020500,10596
...,...,...
68018,72153750501,6113
68019,72153750502,2388
68020,72153750503,2126
68021,72153750601,4397


In [96]:
ACS2019_tract_pop['Tract_pop'].sum()

309832790

In [116]:
# ACS_columns = ['cbg_FIPS', 'cbg_total_population', 'p_asian', 'p_black', 'p_white', 'Shape_Area', 'total_household_income', 'total_households', 'mean_household_income', 'median_household_income']
ACS2019_tract_pop['tract'] = ACS2019_tract_pop['tract'].astype(int)
idx = ACS2019_tract_pop['tract'].astype(str).str.startswith("37")

MSA_pop_df = MSA_df.merge(ACS2019_tract_pop[idx], how='inner', left_on='origin', right_on='tract')#.dropna()
MSA_pop_df = MSA_pop_df.rename({"Tract_pop":"origin_pop"}, axis="columns").drop(columns=['tract'])
MSA_pop_df = MSA_pop_df.merge(ACS2019_tract_pop[idx], how='inner', left_on='destination', right_on='tract')#.dropna()
MSA_pop_df = MSA_pop_df.rename({"Tract_pop":"destination_pop"}, axis="columns").drop(columns=['tract'])
MSA_pop_df.to_csv(f'{MSA_name}_OD_pop.csv', index=False)
MSA_pop_df

,origin,destination,trips,O_lon,O_lat,D_lon,D_lat,geo_distance,origin_pop,destination_pop
0,37119003103,37119002800,788,-80.861828,35.167207,-80.821300,35.192552,4640.554944,4922,4000
1,37119005604,37119002800,155,-80.740552,35.303625,-80.821300,35.192552,14347.965254,5819,4000
2,37097061401,37119002800,86,-80.935275,35.564090,-80.821300,35.192552,42502.219544,5190,4000
3,37159051101,37119002800,126,-80.486895,35.548664,-80.821300,35.192552,49844.686469,5905,4000
4,37097061002,37119002800,2,-80.976474,35.850858,-80.821300,35.192552,74382.899995,4766,4000
...,...,...,...,...,...,...,...,...,...,...
213298,37159051901,37097061101,55,-80.686779,35.680221,-81.052504,35.804651,35847.204190,4505,3240
213299,37119003805,37097061101,3,-80.913097,35.179146,-81.052504,35.804651,70542.847433,3196,3240
213300,37119005918,37097061101,5,-80.979934,35.090514,-81.052504,35.804651,79506.044598,4067,3240
213301,37119004700,37097061101,1,-80.857924,35.244693,-81.052504,35.804651,64585.512759,1789,3240


In [90]:
MSA_df.sort_values('origin')

,origin,destination,trips,O_lon,O_lat,D_lon,D_lat,geo_distance
269969,37007920100,37071032502,6,-79.940439,34.985682,-81.063291,35.184419,104743.074607
86341,37007920100,37179020316,42,-79.940439,34.985682,-80.672034,35.049773,67148.868076
251682,37007920100,37119001916,9,-79.940439,34.985682,-80.705021,35.202824,73761.714206
86342,37007920100,37179021005,106,-79.940439,34.985682,-80.639817,34.923970,64245.699359
219438,37007920100,37159051001,37,-79.940439,34.985682,-80.402420,35.531709,73737.222426
...,...,...,...,...,...,...,...,...
133710,45091061900,37119005910,82,-81.402821,34.941057,-80.998662,35.109532,41348.991824
247941,45091061900,37097061302,8,-81.402821,34.941057,-80.812630,35.670818,97140.246015
261588,45091061900,37119001300,5,-81.402821,34.941057,-80.789047,35.239249,65014.605678
266166,45091061900,37097060801,1,-81.402821,34.941057,-80.765858,36.001844,131124.713485


# Load existing OD population df

In [117]:
MSA_name = r'Charlotte'
OD_pop =  pd.read_csv(f'{MSA_name}_OD_pop.csv')
OD_pop

,origin,destination,trips,O_lon,O_lat,D_lon,D_lat,geo_distance,origin_pop,destination_pop
0,37119003103,37119002800,788,-80.861828,35.167207,-80.821300,35.192552,4640.554944,4922,4000
1,37119005604,37119002800,155,-80.740552,35.303625,-80.821300,35.192552,14347.965254,5819,4000
2,37097061401,37119002800,86,-80.935275,35.564090,-80.821300,35.192552,42502.219544,5190,4000
3,37159051101,37119002800,126,-80.486895,35.548664,-80.821300,35.192552,49844.686469,5905,4000
4,37097061002,37119002800,2,-80.976474,35.850858,-80.821300,35.192552,74382.899995,4766,4000
...,...,...,...,...,...,...,...,...,...,...
213298,37159051901,37097061101,55,-80.686779,35.680221,-81.052504,35.804651,35847.204190,4505,3240
213299,37119003805,37097061101,3,-80.913097,35.179146,-81.052504,35.804651,70542.847433,3196,3240
213300,37119005918,37097061101,5,-80.979934,35.090514,-81.052504,35.804651,79506.044598,4067,3240
213301,37119004700,37097061101,1,-80.857924,35.244693,-81.052504,35.804651,64585.512759,1789,3240


In [47]:
MSA_pop_df.dtypes

origin            int64
destination       int64
trips             int64
O_lon           float64
O_lat           float64
D_lon           float64
D_lat           float64
geo_distance    float64
tract           float64
Tract_pop       float64
dtype: object

In [19]:
MSA_df['origin'] = 

origin            int64
destination       int64
trips             int64
O_lon           float64
O_lat           float64
D_lon           float64
D_lat           float64
geo_distance    float64
dtype: object

Scilit mobility models

https://scikit-mobility.github.io/scikit-mobility/reference/models.html#module-skmob.models.gravity